### Authorship
@author: Alexandre Pereira Santos <br>
alexandre.santos(at)lmu.de<br>
- many scripts in this notebook com from geemap (see below)

### Tasks
- get Sentinel2 data using the Google Earth Engine functionalities
- clip it to an AOI

### Prerequisites
- earth engine api (ee) https://developers.google.com/earth-engine/apidocs
- gee map (gee), https://geemap.org/ by Qiusheng Wu
- create an account and a Google Cloud project in Google Developer, see here: https://developers.google.com/earth-engine/guides/auth

# init

In [71]:
import ee
import geemap
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# extract coordinates from the bounding box
def get_coords(gdf):
    coords = gdf.to_crs(epsg='4326').envelope
    epsg_coords = coords.crs.to_epsg()
    transform = [coords.bounds.values[0][0], coords.bounds.values[0][1], coords.bounds.values[0][2], coords.bounds.values[0][3]]
    roi = ee.Geometry.BBox(west=transform[0], south=transform[1], east=transform[2], north=transform[3])
    rec_roi = ee.Geometry.Rectangle(transform[0],transform[1],transform[2],transform[3])
    coi = roi.centroid(maxError=1)
    return epsg_coords, roi, rec_roi, coi


In [ ]:
# define a function that exports the image to Google Drive
def export_image(image, filename, crs, projection, roi):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=filename,
        folder='data',
        region=roi,
        crs=crs,
        crsTransform=projection['transform'],
        maxPixels=1e13,
        fileFormat='GeoTIFF',
        formatOptions={
            'cloudOptimized': True
        }
    )
    task.start()



# 

In [2]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-alexandresantosgeographie') # Replace with your GEE project
print(ee.__version__)

0.1.389


In [8]:
# defining EE collection
# data from https://gee-community-catalog.org/projects/hand/
hand30_100 = ee.ImageCollection("users/gena/global-hand/hand-100")
hand30_1000 = ee.Image("users/gena/GlobalHAND/30m/hand-1000")
hand90_1000 = ee.Image("users/gena/GlobalHAND/90m-global/hand-1000")
demSRTM = ee.Image("USGS/SRTMGL1_003")

In [11]:
# check if the authentication is working
print(ee.Image("users/gena/GlobalHAND/30m/hand-1000").get("title").getInfo())

None


# imports

In [19]:
#input a vector and a raster file
AOI_path = Path('../../data/processed/')

AOI_file_JAK = 'JAK_LIM_BoundingBox_AOI_A.shp' #Jakarta
AOI_file_MUM = 'MUM_LIM_BoundingBox_AOI_A.shp' #Mumbai
AOI_file_MAN = 'MAN_LIM_BoundingBox_AOI_A.shp' #Manila

#read the vector files
AOI_gdf_JAK = gpd.read_file(AOI_path/AOI_file_JAK).to_crs(epsg=4326)
AOI_gdf_MUM = gpd.read_file(AOI_path/AOI_file_MUM).to_crs(epsg=4326)
AOI_gdf_MAN = gpd.read_file(AOI_path/AOI_file_MAN).to_crs(epsg=4326)

# get the coordinates
JAK_aoi_crs, JAK_roi, JAK_rec_roi, JAK_coi = get_coords(AOI_gdf_JAK)
MUM_aoi_crs, MUM_roi, MUM_rec_roi, MUM_coi = get_coords(AOI_gdf_MUM)
MAN_aoi_crs, MAN_roi, MAN_rec_roi, MAN_coi = get_coords(AOI_gdf_MAN)

#print(coords)

In [ ]:
aoi_coords = AOI_gdf_JAK.to_crs(epsg='4326').envelope
#print('N',aoi_coords.bounds.values[0][3],'S',aoi_coords.bounds.values[0][1],'W',aoi_coords.bounds.values[0][0],'E',aoi_coords.bounds.values[0][2])
transform = [aoi_coords.bounds.values[0][0], aoi_coords.bounds.values[0][1], aoi_coords.bounds.values[0][2], aoi_coords.bounds.values[0][3]]
roi = ee.Geometry.BBox(west=transform[0], south=transform[1], east=transform[2], north=transform[3])
rec_roi = ee.Geometry.Rectangle(transform[0],transform[1],transform[2],transform[3])
coi = roi.centroid(maxError=1)

In [21]:
#code from https://gee-community-catalog.org/projects/hand/
# Add Earth Engine dataset
# This example uses the global HAND model developed by Gennadii Donchyts and colleagues at Deltares.
# The HAND model is a global flood hazard map that combines elevation data with flood simulations. 
# The model is available at 30m and 90m resolution.

Map = geemap.Map()
Map.centerObject(coi, zoom=9)

# Map the function over one year of data and take the median.
# Load Sentinel-2 TOA reflectance data.
collection = hand30_100.map(lambda image: image.clip(roi))

paletteHand = ['023858', '006837', '1a9850', '66bd63', 'a6d96a', 'd9ef8b', 'ffffbf', 'fee08b', 'fdae61', 'f46d43', 'd73027'];
vis = {'min': 1, 'max': 150, 'palette': paletteHand}

# Display the results.
Map.addLayer(hand30_100.mosaic(), vis, 'HAND 100, 30m (~0.1km2)', True)
Map.addLayer(roi, {}, 'ROI')
Map

Map(center=[-6.356096336666425, 106.74822478594142], controls=(WidgetControl(options=['position', 'transparent…

In [32]:
# setting up another import for the HAND model 1000m with 30m resolution
hand30_1000 = ee.Image("users/gena/GlobalHAND/30m/hand-1000")

# the original snippet included this code
# bigFlood = hand30_1000.resample('bicubic').multiply(-1).unitScale(-15, 0).selfMask()
# we calculate flooded areas by subtracting the HAND model from 0 and then rescale the values to a range between 0 and -5
flood = ee.Image.constant(0).subtract(hand30_1000.resample('bicubic'))
flood = flood.unitScale(-5, 0).selfMask()


Map = geemap.Map()
Map.centerObject(MAN_coi, zoom=11)

Map.addLayer(flood, {'min': 0, 'max': 1, 'palette': ['0000ff', '003cd5']}, 'HAND 1000, 30m (~1km2)', True)
#Map.addLayer(roi, {}, 'ROI')
Map


Map(center=[14.499403720751358, 121.06482318543154], controls=(WidgetControl(options=['position', 'transparent…

In [33]:
# create individual images for each of the AOIs
# JAK
JAK_flood = flood.clip(JAK_rec_roi)
JAK_crs='EPSG:'+str(JAK_aoi_crs)
# MUM
MUM_flood = flood.clip(MUM_rec_roi)
MUM_crs='EPSG:'+str(MUM_aoi_crs)
# MAN
MAN_flood = flood.clip(MAN_rec_roi)
MAN_crs='EPSG:'+str(MAN_aoi_crs)

#prepare other vars for export
projection = flood.projection().getInfo()

In [34]:
Map = geemap.Map()
Map.centerObject(MAN_coi, zoom=6)

Map.addLayer(JAK_flood, {'min': 0, 'max': 1, 'palette': ['0000ff', '003cd5']}, 'HAND 1000, 30m (~1km2)', True)
Map.addLayer(MUM_flood, {'min': 0, 'max': 1, 'palette': ['0000ff', '003cd5']}, 'HAND 1000, 30m (~1km2)', True)
Map.addLayer(MAN_flood, {'min': 0, 'max': 1, 'palette': ['0000ff', '003cd5']}, 'HAND 1000, 30m (~1km2)', True)
#Map.addLayer(roi, {}, 'ROI')
Map

Map(center=[14.499403720751358, 121.06482318543154], controls=(WidgetControl(options=['position', 'transparent…

In [36]:

# export the images for the 3 cities

export_image(JAK_flood, 'JAK_HYD_Flood_map_Global_HAND', JAK_crs, projection, JAK_rec_roi)
export_image(MUM_flood, 'MUM_HYD_Flood_map_Global_HAND', MUM_crs, projection, MUM_rec_roi)
export_image(MAN_flood, 'MAN_HYD_Flood_map_Global_HAND', MAN_crs, projection, MAN_rec_roi)